# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,jiagedaqi,50.4167,124.1167,253.82,62,1,1.80,CN,1702358247
1,1,new norfolk,-42.7826,147.0587,295.87,29,87,2.79,AU,1702358248
2,2,piti village,13.4626,144.6933,303.88,76,75,5.14,GU,1702358248
3,3,isafjordur,66.0755,-23.1240,268.06,79,34,3.00,IS,1702358248
4,4,ushuaia,-54.8000,-68.3000,281.96,57,0,1.54,AR,1702358249


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
map_plot_all = city_data_df.hvplot.points("Lng", 
                                        "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )

# Display the map
map_plot_all

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
clean_df = city_data_df.loc[(city_data_df["Max Temp"] < 306)
                                       & (city_data_df["Max Temp"] > 280) 
                                       & (city_data_df["Cloudiness"] < 2)
                                       & (city_data_df["Wind Speed"] > 3)
                                      ]
# Drop any rows with null values
clean_df.dropna()

# Display sample data
clean_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
74,74,sirte,31.2089,16.5887,290.86,73,0,4.53,LY,1702358272
75,75,idri,27.5000,13.2667,283.92,45,0,3.17,LY,1702358272
76,76,la calera,-31.3438,-64.3353,296.62,51,0,5.14,AR,1702358273
170,170,kaya,13.0917,-1.0844,292.05,23,0,5.63,BF,1702358308
178,178,margaret river,-33.9500,115.0667,296.41,58,0,4.73,AU,1702358311


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
74,sirte,LY,31.2089,16.5887,73,
75,idri,LY,27.5000,13.2667,45,
76,la calera,AR,-31.3438,-64.3353,51,
170,kaya,BF,13.0917,-1.0844,23,
178,margaret river,AU,-33.9500,115.0667,58,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel", "apiKey": geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
sirte - nearest hotel: فندق المدينة-سرت
idri - nearest hotel: No hotel found
la calera - nearest hotel: No hotel found
kaya - nearest hotel: Hôtel Kaziendé
margaret river - nearest hotel: Margaret River Hotel
tazacorte - nearest hotel: App Leyma
bardai - nearest hotel: No hotel found
punta arenas - nearest hotel: Hotel Lacolet
carnarvon - nearest hotel: No hotel found
ayorou - nearest hotel: Hotel Aminokal
nguigmi - nearest hotel: Guest PAM
san rafael del sur - nearest hotel: Barceló Montelimar
kidal - nearest hotel: No hotel found
chamical - nearest hotel: No hotel found
port macquarie - nearest hotel: The Observatory
kruisfontein - nearest hotel: No hotel found
garachico - nearest hotel: Residencia El Drago
gao - nearest hotel: No hotel found
gorom-gorom - nearest hotel: No hotel found
luderitz - nearest hotel: Seaview Hotel Zum Sperrgebiet
gustavia - nearest hotel: Hôtel La Presqu'ile
bogande - nearest hotel: No hotel found
upington - nearest hotel: Die Eiland 

,City,Country,Lat,Lng,Humidity,Hotel Name
74,sirte,LY,31.2089,16.5887,73,فندق المدينة-سرت
75,idri,LY,27.5000,13.2667,45,No hotel found
76,la calera,AR,-31.3438,-64.3353,51,No hotel found
170,kaya,BF,13.0917,-1.0844,23,Hôtel Kaziendé
178,margaret river,AU,-33.9500,115.0667,58,Margaret River Hotel
201,tazacorte,ES,28.6290,-17.9293,45,App Leyma
228,bardai,NG,11.6500,7.2333,43,No hotel found
267,punta arenas,CL,-53.1500,-70.9167,76,Hotel Lacolet
270,carnarvon,AU,-24.8667,113.6333,38,No hotel found
295,ayorou,NE,14.7308,0.9174,26,Hotel Aminokal


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
hotels_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.5,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotels_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)